--- Simple aggregations

1--> will give you only one output per row
coun the total number of records, you want to find the sum of quantities

count the total number of records, count number of distance invoice ids, sum of quantities, avg unit price

2--> group aggregations
-- we will do a group by



In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/public/trendytech/datasets/order_data.csv")

In [3]:
orders_df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536378|     null|PACK OF 60 DINOSA...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|     null|PACK OF 60 PINK P...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|    84991|60 TEATIME FAIRY ...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|   84519A|TOMATO CHARLIE+LO...|       6|01-12-2010 9.37|     2.95|     14688|United Kingdom|
|   536378|   85183B|CHARLIE & LOLA WA...|      48|01-12-2010 9.37|     1.25|     14688|United Kingdom|
|   536378|   85071B|RED CHARLIE+LOLA ...|      96|01-12-2010 9.37|     0.38|     14688|United Kingdom|
|   536378|    21931|JUMBO STORAGE BAG...|      10|01-12-2010 9.

In [4]:
orders_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
from pyspark.sql.functions import *

In [10]:
orders_df.select(count("*").alias("row_count"),countDistinct("InvoiceNo").alias("unique_invoiuce"),sum("Quantity").alias("total_quantity"),avg("UnitPrice").alias("avg_price")).show()

+---------+---------------+--------------+----------------+
|row_count|unique_invoiuce|total_quantity|       avg_price|
+---------+---------------+--------------+----------------+
|   541782|          25858|       5175855|4.61156532332193|
+---------+---------------+--------------+----------------+



In [8]:
orders_df.select(count("*").alias("row_count"),countDistinct("InvoiceNo").alias("unique_invoice"),sum("Quantity").alias("total_quantity"),avg("UnitPrice").alias("avg_price")).show()

+---------+--------------+--------------+----------------+
|row_count|unique_invoice|total_quantity|       avg_price|
+---------+--------------+--------------+----------------+
|   541782|         25858|       5175855|4.61156532332193|
+---------+--------------+--------------+----------------+



In [11]:
orders_df.selectExpr("count(*) as row_count", "count(distinct(invoiceno)) as unique_invoice","sum(Quantity) as total_quantity","avg(unitprice) as avg_price").show()

+---------+--------------+--------------+-----------------+
|row_count|unique_invoice|total_quantity|        avg_price|
+---------+--------------+--------------+-----------------+
|   541782|         25858|       5175855|4.611565323321925|
+---------+--------------+--------------+-----------------+



count the total number of records, count number of distance invoice ids, sum of quantities, avg unit price

In [14]:
orders_df.createOrReplaceTempView("orders")

In [15]:
spark.sql(" select count(*) as row_count, count(distinct(InvoiceNo)), sum(Quantity) as total_quantity, avg(unitprice) as avg_price from orders").show()

+---------+-------------------------+--------------+-----------------+
|row_count|count(DISTINCT InvoiceNo)|total_quantity|        avg_price|
+---------+-------------------------+--------------+-----------------+
|   541782|                    25858|       5175855|4.611565323321931|
+---------+-------------------------+--------------+-----------------+

